In [0]:
import os

# Set schemas and volume path
catalog = "edl_dev_ctlg"
source_schema = "structured"
target_schema = "structured"
volume_path = "/Volumes/edl_dev_ctlg/rawfiles/raw/EDF/dab"
table_suffix = "_c73550"
limit = 100
# Get tables
tables = [
    row.tableName
    for row in spark.sql(f"SHOW TABLES IN {catalog}.{source_schema}").collect()
    if row.tableName.startswith("dlt_meta") and not row.tableName.endswith(table_suffix) 
]

create_statements = []
insert_statements = []

for table in tables:
    full_source_table = f"{catalog}.{source_schema}.{table}"
    full_target_table = f"{catalog}.{target_schema}.{table}"

    df = spark.table(full_source_table)
    schema = df.schema

    columns_def = [
        f"`{field.name}` {field.dataType.simpleString().lower()}"
        for field in schema.fields
    ]
    columns_sql = ",\n  ".join(columns_def)

    create_sql = f"""CREATE TABLE IF NOT EXISTS {full_target_table} ({columns_sql});"""
    # insert_sql = f"""INSERT INTO {full_target_table} SELECT * FROM {full_source_table} limit {limit};"""

    create_statements.append(create_sql)
    # -- INSERT INTO ... VALUES
    data_rows = df.limit(limit).collect()
    if not data_rows:
        continue
    column_names = [f"`{f.name}`" for f in schema.fields]
    col_list = ", ".join(column_names)

    value_lines = []
    for row in data_rows:
        values = []
        for val in row:
            if val is None:
                values.append("NULL")
            elif isinstance(val, str):
                # Escape single quotes
                values.append(f"""'{val.replace("'", "''")}'""")
            elif isinstance(val, bool):
                values.append("TRUE" if val else "FALSE")
            else:
                values.append(str(val))
        value_lines.append(f"({', '.join(values)})")

    values_sql = ",\n".join(value_lines)
    insert_sql = f"""INSERT INTO {full_target_table} ({col_list})VALUES{values_sql};"""
    insert_statements.append(insert_sql)


# Combine scripts
create_script = "-- CREATE TABLE SCRIPTS --\n\n" + "\n\n".join(create_statements)
insert_script = "-- INSERT SCRIPTS --\n\n" + "\n\n".join(insert_statements)

# Write to files in the Volume
create_file_path = os.path.join(volume_path, "create_tables.sql")
insert_file_path = os.path.join(volume_path, "insert_data.sql")

# Save using Python file write (ensure it's running on Driver)
with open(create_file_path, "w") as f:
    f.write(create_script)

with open(insert_file_path, "w") as f:
    f.write(insert_script)

print(f"✅ Scripts saved to:\n- {create_file_path}\n- {insert_file_path}")

✅ Scripts saved to:
- /Volumes/edl_dev_ctlg/rawfiles/raw/EDF/dab/create_tables.sql
- /Volumes/edl_dev_ctlg/rawfiles/raw/EDF/dab/insert_data.sql
